In [1]:
%load_ext autoreload
%autoreload 2
import os
if not os.path.exists('./modules') and not os.path.exists('modules.zip'):
    from google.colab import files
    uploaded = files.upload()
if not os.path.exists('./modules') and os.path.exists('modules.zip'):
    os.system('unzip modules.zip -d .')

!pip3 install optuna
import torch
import torch.nn as nn
import optuna
from modules import Trainer
from modules.competition_dataset import EEGDataset
from modules.utils import split_and_get_loaders, evaluate_model, get_closest_divisor
import matplotlib.pyplot as plt
import random
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Saving modules.zip to modules.zip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 23.7 MB/s eta 0:00:00


device(type='cuda')

In [20]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/ai_data/eeg_detection/data/mtcaic3'
model_path = '/content/drive/MyDrive/ai_data/eeg_detection/checkpoints/ssvep/models/ssvep.pth'
optuna_db_path = '/content/drive/MyDrive/ai_data/eeg_detection/checkpoints/ssvep/optuna/optuna_studies.db'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call this function before creating datasets and models
set_random_seeds(42)

In [4]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class DepthWiseConv2D(nn.Module):
    def __init__(self, in_channels, kernel_size, dim_mult=1, padding=0, bias=False):
        super(DepthWiseConv2D, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels * dim_mult, padding=padding, kernel_size=kernel_size, groups=in_channels, bias=bias)

    def forward(self, x: torch.Tensor):
        return self.depthwise(x)


class SeperableConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, bias=False):
        super(SeperableConv2D, self).__init__()
        self.depthwise = DepthWiseConv2D(in_channels, kernel_size, padding=padding)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

class SSVEPClassifier(nn.Module):
    # EEG Net Based
    # todo look at this https://paperswithcode.com/paper/a-transformer-based-deep-neural-network-model
    def __init__(self, n_electrodes=16, n_samples=128, out_dim=4, dropout=0.25, kernLength=256, F1=96, D=1, F2=96, hidden_dim=100, layer_dim=1):
        super().__init__()

        # B x C x T
        self.block_1 = nn.Sequential(
            nn.Conv2d(1, F1, (1, kernLength), padding='same', bias=False),
            nn.BatchNorm2d(F1),
            #
            DepthWiseConv2D(F1, (n_electrodes, 1), dim_mult=D, bias=False),
            nn.BatchNorm2d(F1*D),
            nn.ELU(),
            nn.MaxPool2d((1, 2)), # todo try making this max pool
            nn.Dropout(dropout),
            #
            SeperableConv2D(F1 * D, F2, kernel_size=(1, 16), padding='same', bias=False),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.MaxPool2d((1, 4)),
            nn.Dropout(dropout),
        )

        self.lstm_head = LSTMModel(F2, hidden_dim, layer_dim, out_dim)

    def forward(self, x: torch.Tensor):
        """expected input shape: BxCxT"""
        x = x.unsqueeze(1)
        y = self.block_1(x) # B x F1 x 1 x time_sub

        y = y.squeeze(2) # B x F1 x time_sub
        y = y.permute(0, 2, 1) # B x time_sub x F1
        y = self.lstm_head(y)

        return y


dummy_x = torch.randn(5, 14, 320)
model = SSVEPClassifier(n_electrodes=dummy_x.shape[1], n_samples=dummy_x.shape[2])
model(dummy_x)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /pytorch/aten/src/ATen/native/Convolution.cpp:1036.)
  return F.conv2d(


tensor([[-0.2054, -0.0184,  0.1601,  0.0599],
        [-0.1972,  0.1410,  0.1540,  0.0297],
        [-0.2554, -0.0320, -0.0062,  0.0351],
        [-0.1913,  0.0777,  0.0951,  0.0720],
        [-0.2877,  0.0788,  0.1244,  0.0170]], grad_fn=<AddmmBackward0>)

In [ ]:
window_length = get_closest_divisor(160)
print(window_length)
stride = window_length // 3
batch_size = 64

dataset = EEGDataset(data_path, window_length=window_length, stride=stride)
train_loader, val_loader, test_loader = split_and_get_loaders(dataset, batch_size)

In [6]:
model = SSVEPClassifier(
    n_electrodes=dummy_x.shape[1],
    n_samples=dummy_x.shape[2],
    dropout=0.33066508963955576,
    kernLength=256,
    F1 = 128,
    D = 2,
    F2 = 96,
    hidden_dim=256,
    layer_dim=3,
).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.00030241790493218325)
avg_losses = []
val_accuracies = []

epochs = 200
for epoch in range(epochs):
    avg_loss = 0
    model.train()
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x).to(device)

        loss = criterion(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        avg_loss += loss.item()

    avg_loss /= len(train_loader)
    avg_losses.append(avg_loss)

    evaluation = evaluate_model(model, val_loader, device)
    val_accuracies.append(evaluation)
    print(f'epoch: {epoch}, avg_loss: {avg_loss}, val_evaluation: {evaluation}')

In [ ]:
# maxpool
plt.plot(range(len(avg_losses)), avg_losses, "b-", label="trainingg loss")
plt.plot(range(len(val_accuracies)), val_accuracies, "r-", label="validation accuracies")
plt.legend()
print(f"min avg_losses: {min(avg_losses)}")
print(f"max val_accuracies: {max(val_accuracies)}")

In [21]:
batch_size = 64
window_length = 175
stride_factor=3

In [ ]:
class CustomTrainer(Trainer):
    def _prepare_training(self, is_trial, do_not_modify_network=True, batch_size=batch_size, window_length=window_length, stride_factor=stride_factor):
        super()._prepare_training(is_trial, do_not_modify_network, batch_size=batch_size,
                                  window_length=window_length,
                                  stride_factor=stride_factor)
        assert self.dataset is not None

        if is_trial:
            assert isinstance(self.trial, optuna.Trial), "trial is none, cant' suggest params"

            if do_not_modify_network:
                best_params = self._get_study().best_params if do_not_modify_network else None
                assert best_params is not None, "best_params is None, can't use them"

                kernLength = best_params["kernLength"]
                F1 = best_params["F1"]
                D = best_params["D"]
                F2 = best_params["F2"]
                hidden_dim = best_params["hidden_dim"]
                layer_dim = best_params["layer_dim"]

            else:
                kernLength = self.trial.suggest_categorical("kernLength", [128, 256, 512])
                F1 = self.trial.suggest_categorical("F1", [64, 96, 128])
                D = self.trial.suggest_categorical("D", [1, 2, 3])
                F2 = self.trial.suggest_categorical("F2", [64, 96, 128])
                hidden_dim = self.trial.suggest_categorical("hidden_dim", [64, 128, 256])
                layer_dim = self.trial.suggest_categorical("layer_dim", [1, 2, 3, 4])

            dropout = self.trial.suggest_float("dropout", 0, 0.5)
            lr = self.trial.suggest_float("lr", 3e-4, 3e-2, log=True)

        else:
            # best_params = self._get_study().best_params
            # kernLength = best_params["kernLength"]
            # F1 = best_params["F1"]
            # D = best_params["D"]
            # F2 = best_params["F2"]
            # hidden_dim = best_params["hidden_dim"]
            # layer_dim = best_params["layer_dim"]
            # dropout = best_params["dropout"]
            # lr = best_params["lr"]
            dropout=0.33066508963955576
            kernLength=256
            F1 = 128
            D = 2
            F2 = 96
            hidden_dim=256
            layer_dim=3
            lr = 0.000010241790493218325

        n_samples = self.dataset.data[0].shape[1]  # data[x] shape CxT
        n_electrodes = self.dataset.data[0].shape[0]

        n_samples = self.dataset.data[0].shape[1]  # data[x] shape CxT
        n_electrodes = self.dataset.data[0].shape[0]

        self.model = SSVEPClassifier(
            n_electrodes=n_electrodes, n_samples=n_samples, out_dim=4, dropout=dropout, kernLength=kernLength, F1=F1, D=D, F2=F2, hidden_dim=hidden_dim, layer_dim=layer_dim
        )
        print(f"lr: {lr}")
        try:
            self.model.load_state_dict(torch.load(model_path))
            print(f"loaded model weights")
        except Exception:
            print(f"no model weights found at {model_path}")
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)

trainer = CustomTrainer(data_path, optuna_db_path, model_path, train_epochs=10000, optuna_n_trials=35)

In [40]:
delete_existing = False
trainer.optimize(delete_existing)

[I 2025-06-20 11:16:10,045] Using an existing study with name 'ssvep_classifier_optimization' instead of creating a new one.
[I 2025-06-20 11:16:10,152] Using an existing study with name 'ssvep_classifier_optimization' instead of creating a new one.
[W 2025-06-20 11:16:10,198] Trial 15 failed with parameters: {} because of the following error: AssertionError("widnow length 160 doesn't divide trial length 1750").
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/content/modules/trainer.py", line 97, in _objective
    self._prepare_training(True)
  File "/tmp/ipython-input-39-138637841.py", line 3, in _prepare_training
    super()._prepare_training(is_trial, do_not_modify_network, batch_size=batch_size,
  File "/content/modules/trainer.py", line 92, in _prepare_training
    self.dataset = EEGDataset(data_path=self.data_path, win

AssertionError: widnow length 160 doesn't divide trial length 1750

In [41]:
# manual_write_study_params(trainer.study_name, trainer.storage)
trainer.train()

[I 2025-06-20 11:16:11,971] Using an existing study with name 'ssvep_classifier_optimization' instead of creating a new one.


lr: 1.0241790493218325e-05
loaded model weights
epoch 0, evaluation 0.74873046875, avg_loss 0.30769792664796114
epoch 1, evaluation 0.74873046875, avg_loss 0.30357948478776964
epoch 2, evaluation 0.74990234375, avg_loss 0.3023449623025954
epoch 3, evaluation 0.7490234375, avg_loss 0.30571578615345063
epoch 4, evaluation 0.750390625, avg_loss 0.30304136825725436
epoch 5, evaluation 0.7505859375, avg_loss 0.3014926631003618
epoch 6, evaluation 0.75224609375, avg_loss 0.2968446263577789
epoch 7, evaluation 0.7498046875, avg_loss 0.3004373211879283
epoch 8, evaluation 0.75087890625, avg_loss 0.29997859699651597
epoch 9, evaluation 0.75185546875, avg_loss 0.29749580766074357
epoch 10, evaluation 0.7515625, avg_loss 0.2988324134144932
epoch 11, evaluation 0.7498046875, avg_loss 0.2982925652177073
epoch 12, evaluation 0.7490234375, avg_loss 0.29697722850833086
epoch 13, evaluation 0.74951171875, avg_loss 0.29678182139759884
epoch 14, evaluation 0.75, avg_loss 0.29814286148175595
epoch 15, eva

KeyboardInterrupt: 

In [ ]:
trainer._prepare_training(False)
f"test accuracy: {evaluate_model(trainer.model, trainer.test_loader, device)}"

[I 2025-06-20 12:07:47,568] Using an existing study with name 'ssvep_classifier_optimization' instead of creating a new one.
[I 2025-06-20 12:07:56,462] Using an existing study with name 'ssvep_classifier_optimization' instead of creating a new one.


loaded model weights from ./checkpoints/ssvep/models/70_lstm_ssvep.pth


'test accuracy: 0.6919157608695652'

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

y_true = []
y_pred = []

trainer.model.eval()
with torch.no_grad():
    for x, y in trainer.val_loader:
        x = x.to(device)
        outputs = trainer.model(x)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(y.numpy())

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.70      0.69      1221
           1       0.71      0.68      0.69      1134
           2       0.70      0.70      0.70      1149
           3       0.70      0.70      0.70      1168

    accuracy                           0.69      4672
   macro avg       0.70      0.69      0.69      4672
weighted avg       0.69      0.69      0.69      4672

